In [2]:
# Somehow in mac, it cant find local packages
# Run this to solve no local modules found in mac
import os
import sys

# Get the project root directory (assuming your notebook is in a subdirectory)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


# Python lib
import asyncio
import os

# Autogen-0.4
from autogen_agentchat.agents import AssistantAgent

from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Local
from utils import get_dataset_profile, get_columns_sample, initialize_individual_chat, jsonify_prompt
from prompts import data_dict_summarizer_prompt

In [5]:
root = '../sheets/mysql/'
files: list[str] = os.listdir(path=root)

# Uses utils/get_dataset_profile
# tasks = [get_dataset_profile(root, file_name = file, output_format='json') for file in files]

# # Uses utils/get_columns_sample
tasks = [get_columns_sample(root, file_name = file) for file in files]

results = await asyncio.gather(*tasks)
result_dict = zip(files, results)
results = list(result_dict)

In [37]:
data_dict_summarizer = "qwen2.5:32b-instruct-q8_0-32768"
data_dict_generator = "qwen2.5:32b-instruct-q8_0"

# Common config
llm_base_url = "http://34.204.63.234:11434/v1"
api_key = "none"
model_info =  {
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "family": "Qwen2.5"
    }

#######################################################################
#   !!! DONT EDIT BELOW EXCEPT FOR if __name__ == "__main__":   !!!   #
#######################################################################


# Create reasoning config
# Need to be more creative
data_dict_summarizer_client = OpenAIChatCompletionClient(
    frequency_penalty=0.2, 
    logit_bias=None, 
    max_tokens=128000,
    presence_penalty=0.5, 
    response_format={"type": "json_object"}, 
    seed=42, 
    temperature=0.8, 
    top_p=0.95, 
    model=data_dict_summarizer,
    api_key='none',  
    model_info=model_info, 
    base_url='http://34.204.63.234:11434/v1'
    )

# Just generate the data dict based on obervered facts
data_dict_generator_client = OpenAIChatCompletionClient(
    frequency_penalty=0.4, 
    logit_bias=None, 
    max_tokens=2048, 
    presence_penalty=0.3, 
    response_format={"type": "json_object"},
    seed=42, 
    temperature=0.2, 
    top_p=0.7, 
    model=data_dict_generator,
    api_key='none', 
    model_info=model_info, 
    base_url='http://34.204.63.234:11434/v1')



In [13]:
# Create a list of tasks for all agents
tasks = [initialize_individual_chat(filename=results[index][0], metadata=results[index][1], data_dict_generator_client=data_dict_generator_client) for index in range(len(results))]

# Execute all agent chat tasks concurrently
responses = await asyncio.gather(*tasks)
print(f"\nResponses received: {len(responses)}")
print(responses)



[Response(chat_message=TextMessage(source='File_handler_customer_csv', models_usage=RequestUsage(prompt_tokens=867, completion_tokens=1481), content='<think>\nAlright, I need to create a data dictionary for the customer.csv file based on the provided metadata. Let me go through each column step by step.\n\nFirst, looking at \'customerNumber\'. The sample values are \'103\', \'112\', and \'114\'. These look like identifiers, probably numeric codes without any specific pattern beyond being integers. So I\'ll note it as an Identifier format with nullable set to False since there\'s no null in the samples.\n\nNext is \'customerName\' with names like \'Atelier graphique\' and others. This is clearly text data, so the Format should be Text. It doesn\'t allow nulls based on the examples provided.\n\nMoving on to \'contactLastName\', these are last names such as \'Schmitt\'. Again, this is textual information, so I\'ll categorize it under Text format with nullable False.\n\nFor \'contactFirstN

In [24]:
# Extract and transform text content into json, also return filepath
responses_TextMessage, filepath = jsonify_prompt(responses)

[{'Data Dictionary': {'filename': 'customer.csv',
   'columns': [{'Column': 'customerNumber',
     'Description': 'Unique identifier for each customer.',
     'Format': 'Identifier: Starts with a number like 103, 112, etc.',
     'Nullable': False,
     'Sample Values': ['103', '112', '114']},
    {'Column': 'customerName',
     'Description': 'The full name of the customer or company.',
     'Format': "Text: Contains free-form text strings like 'Atelier graphique'.",
     'Nullable': False,
     'Sample Values': ['Atelier graphique',
      'Signal Gift Stores',
      'Australian Collectors, Co.']},
    {'Column': 'contactLastName',
     'Description': 'The last name of the primary contact person for the customer.',
     'Format': "Text: Contains free-form text strings like 'Schmitt'.",
     'Nullable': False,
     'Sample Values': ['Schmitt', 'King', 'Ferguson']},
    {'Column': 'contactFirstName',
     'Description': 'The first name of the primary contact person for the customer.',
 

In [25]:
# Construct a list of TextMessage as prompt
# Combine data dict ingestion agent prompt with all previous responses
initial_task = [TextMessage(content=f"Review the data dictionary generated by the file handling agents.",source="user")]
responses_TextMessage.extend(initial_task)

In [26]:
responses_TextMessage

[{'Data Dictionary': {'filename': 'customer.csv',
   'columns': [{'Column': 'customerNumber',
     'Description': 'Unique identifier for each customer.',
     'Format': 'Identifier: Starts with a number like 103, 112, etc.',
     'Nullable': False,
     'Sample Values': ['103', '112', '114']},
    {'Column': 'customerName',
     'Description': 'The full name of the customer or company.',
     'Format': "Text: Contains free-form text strings like 'Atelier graphique'.",
     'Nullable': False,
     'Sample Values': ['Atelier graphique',
      'Signal Gift Stores',
      'Australian Collectors, Co.']},
    {'Column': 'contactLastName',
     'Description': 'The last name of the primary contact person for the customer.',
     'Format': "Text: Contains free-form text strings like 'Schmitt'.",
     'Nullable': False,
     'Sample Values': ['Schmitt', 'King', 'Ferguson']},
    {'Column': 'contactFirstName',
     'Description': 'The first name of the primary contact person for the customer.',
 

In [ ]:
# Initialize data dict ingestion agent
final_agent = AssistantAgent(name="Data_Dictionary_Analytics_Suggester",
                             model_client=data_dict_summarizer_client,
                             system_message=data_dict_summarizer_prompt()
)

response = await final_agent.on_messages(
        [TextMessage(content=f"{responses_TextMessage}\n\nReview the above data dictionary.\n\n{responses_TextMessage}",source="user")], None
    )
response

Response(chat_message=TextMessage(source='Data_Dictionary_Analytics_Suggester', models_usage=RequestUsage(prompt_tokens=9374, completion_tokens=3767), content="### Use Case 1\n- **Use Case ID:** C001\n- **Title:** Customer Payment Analysis\n- **Description:** Analyze customer payment patterns to identify high-paying customers and evaluate their contribution to overall revenue. This can help in understanding which customers are valuable for targeted marketing campaigns.\n- **Data Sources:**\n    - `customer.csv`\n    - `payments.csv`\n- **Columns Utilized:**\n    - `customer.csv.customerNumber`\n    - `customer.csv.customerName`\n    - `payments.csv.customerNumber`\n    - `payments.csv.paymentDate`\n    - `payments.csv.amount`\n- **Relationships Leveraged:** \n    - The `customerNumber` in both datasets allows linking customer details with their payment history.\n    \n---\n\n### Use Case 2\n- **Use Case ID:** C002\n- **Title:** Sales Representative Performance Evaluation\n- **Descripti

In [28]:
print(response.chat_message.content)

### Use Case 1
- **Use Case ID:** C001
- **Title:** Customer Payment Analysis
- **Description:** Analyze customer payment patterns to identify high-paying customers and evaluate their contribution to overall revenue. This can help in understanding which customers are valuable for targeted marketing campaigns.
- **Data Sources:**
    - `customer.csv`
    - `payments.csv`
- **Columns Utilized:**
    - `customer.csv.customerNumber`
    - `customer.csv.customerName`
    - `payments.csv.customerNumber`
    - `payments.csv.paymentDate`
    - `payments.csv.amount`
- **Relationships Leveraged:** 
    - The `customerNumber` in both datasets allows linking customer details with their payment history.
    
---

### Use Case 2
- **Use Case ID:** C002
- **Title:** Sales Representative Performance Evaluation
- **Description:** Evaluate the performance of sales representatives by analyzing their associated customers' orders and payments. This can help identify top-performing sales reps who contribute